In [1]:
#第8章 数据规整：聚合、合并和重塑
#在许多应用中，数据可能分散在许多文件或者数据库中，存储的形式也不利于分析。
#本章关注可以聚合、合并、重塑数据的方法

In [2]:
#8.1层次化索引
#层次化索引式pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低纬度形式处理高维度数据
#创建一个Series，并用一个列表或数组组成的列表作为索引
import pandas as pd
import numpy as np
data=pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

In [3]:
data

a  1   -0.670191
   2   -0.708757
   3   -1.144294
b  1   -0.179574
   3    1.994843
c  1    1.309567
   2   -1.700986
d  2   -0.215767
   3    0.379632
dtype: float64

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [5]:
#对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单
data['b']

1   -0.179574
3    1.994843
dtype: float64

In [6]:
data['b':'c']

b  1   -0.179574
   3    1.994843
c  1    1.309567
   2   -1.700986
dtype: float64

In [7]:
data['b':'d']

b  1   -0.179574
   3    1.994843
c  1    1.309567
   2   -1.700986
d  2   -0.215767
   3    0.379632
dtype: float64

In [8]:
data.loc[['b','d']]

b  1   -0.179574
   3    1.994843
d  2   -0.215767
   3    0.379632
dtype: float64

In [9]:
#有时甚至还可以在‘内层’中进行选取
data.loc[:,2]

a   -0.708757
c   -1.700986
d   -0.215767
dtype: float64

In [10]:
#层次化索引在数据重塑和基于分组的操作（如透视图生成）中扮演者重要的角色。
#例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中：
data.unstack()

,1,2,3
a,-0.670191,-0.708757,-1.144294
b,-0.179574,NaN,1.994843
c,1.309567,-1.700986,NaN
d,NaN,-0.215767,0.379632


In [11]:
#unstack的逆运算是stack:
data.unstack().stack()

a  1   -0.670191
   2   -0.708757
   3   -1.144294
b  1   -0.179574
   3    1.994843
c  1    1.309567
   2   -1.700986
d  2   -0.215767
   3    0.379632
dtype: float64

In [12]:
#对于一个DataFrame，每条轴都可以有分层索引:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),index=[['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])
                                                                                        

In [13]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [14]:
#各层都可以有名字，如果指定了名称，他们就会显示在控制台输出中：
frame.index.names=['key1','key2']

In [15]:
frame.columns.names=['state','color']

In [16]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
#有了部分列索引，因此可以轻松选取列分组：
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [18]:
#重排与分级排序
#有时候需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。

In [19]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [20]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [22]:
#根据级别汇总统计
#许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于指定在某条轴上求和的级别。
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [23]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [24]:
#使用DataFrame的列进行索引
frame=pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],'d':[0,1,2,0,1,2,3]})

In [25]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [26]:
#DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame
frame2=frame.set_index(['c','d'])

In [27]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [28]:
#默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来
frame.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [29]:
#reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移列里面
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [30]:
#8.2合并数据集
#pandas对象中的数据可以通过一些方式合并
#数据集的合并（merge）或连接（join）运算是通过一个或多个键将行连接起来的。
df1=pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})

In [31]:
df2=pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [32]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [33]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [34]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [35]:
#如果两个对象的列名不同，也可以分别进行指定
df3=pd.DataFrame({'1key':['b','b','a','c','a','a','b'],'data1':range(7)})

In [36]:
df4=pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})

In [37]:
pd.merge(df3,df4,left_on='1key',right_on='rkey')

,1key,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [38]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [39]:
#多对多的合并有些不直观。
df1=pd.DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})

In [40]:
df2=pd.DataFrame({'key':['a','b','a','b','d'],'data2':range(5)})

In [41]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [42]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [43]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [44]:
#多对多连接产生的是行的笛卡尔积。由于左边的DataFrame有3个'b'行，右边的有2个。所以最终结果中就有6个’b‘行。连接方式只影响出现在结果中的不同键的值
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [45]:
df2['Data3']=None

In [46]:
df2

,key,data2,Data3
0,a,0,None
1,b,1,None
2,a,2,None
3,b,3,None
4,d,4,None


In [47]:
#多对多连接产生的是行的笛卡尔积。由于左边的DataFrame有3个‘b’行，右边的有2个，所以最终结果中就有6个‘b’行。连接方式
#只影响出现在结果中的不同的键的值
pd.merge(df1,df2,how='inner')

,key,data1,data2,Data3
0,b,0,1,None
1,b,0,3,None
2,b,1,1,None
3,b,1,3,None
4,b,5,1,None
5,b,5,3,None
6,a,2,0,None
7,a,2,2,None
8,a,4,0,None
9,a,4,2,None


In [48]:
#要根据多个键进行合并，传入一个由列名组成的列表即可：
left=pd.DataFrame({'key1':['foo','foo','bar'],'key2':['one','two','one'],'lval':[1,2,3]})
right=pd.DataFrame({'key1':['foo','foo','bar','bar'],'key2':['one','one','one','two'],'rval':[4,5,6,7]})
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [49]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [50]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [51]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [52]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [53]:
#索引上的合并
left1=pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})

In [54]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [55]:
right1=pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])

In [56]:
right1

,group_val
a,3.5
b,7.0


In [57]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [58]:
#对于层次化索引的数据，事情就有点复杂了，因为索引的合并默认是多键合并：
lefth=pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],'key2':[2000,2001,2002,2001,2002],'data':np.arange(5.)})

In [59]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [60]:
righth=pd.DataFrame(np.arange(12).reshape((6,2)),index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],[2001,2000,2000,2000,2001,2002]],
                   columns=['event1','event2'])

In [61]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [62]:
#这种情况下，必须以列表的形式指明用作合并键的多个列（注意用how=‘outer’对重复索引值的处理）：
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [63]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [64]:
#同时使用合并双方的索引也没问题：
left2=pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],index=['a','c','e'],columns=['Ohio','Nevada'])

In [65]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [66]:
right2=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[12,14]],index=['b','c','d','e'],columns=['Missouri','Alabama'])

In [67]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,12.0,14.0


In [68]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,12.0,14.0


In [69]:
#DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并。
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,12.0,14.0


In [70]:
#DataFrame的join方法默认使用的是左连接，保留左边表的行索引。它还支持在调用的DataFrame的列上，连接传递的DataFrame索引
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [71]:
#对于简单的索引合并，你还可以向join传入一组DataFrame，下一节会介绍更为通用的concat函数，也能实现此功能
anothor=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],index=['a','c','e','f'],columns=['New York','Oregon'])

In [72]:
anothor

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [73]:
left2.join([right2,anothor])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,12.0,14.0,11.0,12.0


In [74]:
#轴向连接
#另一种数据合并运算也被称作连接，绑定或堆叠。NumPy的concatenation函数可以用NumPy数组来做：
arr=np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [75]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [76]:
#对于pandas对象，带有标签的轴使你能够进一步推广数组的连接运算。
#如果对象在其他轴上的索引不同，我们应该合并这些轴的不同元素还是只使用交集？
s1=pd.Series([0,1],index=['a','b'])
s2=pd.Series([2,3,4],index=['c','d','e'])
s3=pd.Series([5,6],index=['f','g'])

In [77]:
#对这些对象调用concat可以将值和索引粘合在一起
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [78]:
pd.concat([s1,s2,s3],axis=1)

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [79]:
#另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来。传入join=‘inner’即可得到他们的交集
s4=pd.concat([s1,s3])

In [80]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [81]:
pd.concat([s1,s4],axis=1)

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [82]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


In [83]:
#通过join_axes指定要在其他轴上使用的索引：
pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [84]:
result=pd.concat([s1,s1,s3],keys=['one','two','three'])

In [85]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [86]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [87]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [88]:
df1=pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])

In [89]:
df2=pd.DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])

In [90]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [91]:
df2

,three,four
a,5,6
c,7,8


In [92]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [93]:
#如果传入的不是一个列表而是一个字典，则字典的键就会被当作keys选项的值
pd.concat({'level1':df1,'level2':df2},axis=1)

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [94]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'],names=['upper','lower'])

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [95]:
#关于DataFrame的问题是，DataFrame的行索引不包含任何相关数据：
df1=pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])

In [96]:
df1

,a,b,c,d
0,-0.039256,1.351152,-0.956674,0.942033
1,0.571195,-0.546405,-0.732922,0.319301
2,0.550157,0.205131,-0.182282,-0.684912


In [97]:
df2=pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])

In [98]:
df1

,a,b,c,d
0,-0.039256,1.351152,-0.956674,0.942033
1,0.571195,-0.546405,-0.732922,0.319301
2,0.550157,0.205131,-0.182282,-0.684912


In [99]:
df2

,b,d,a
0,-1.069789,0.511598,0.372596
1,-0.761702,0.473786,0.835811


In [100]:
pd.concat([df1,df2],ignore_index=True)

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.039256,1.351152,-0.956674,0.942033
1,0.571195,-0.546405,-0.732922,0.319301
2,0.550157,0.205131,-0.182282,-0.684912
3,0.372596,-1.069789,NaN,0.511598
4,0.835811,-0.761702,NaN,0.473786


In [101]:
#合并重叠数据
a=pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index=['f','e','d','c','b','a'])

In [102]:
b=pd.Series(np.arange(len(a),dtype=np.float64),index=['f','e','d','c','b','a'])

In [103]:
b[-1]=np.nan

In [104]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [105]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [106]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [107]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [108]:
df1=pd.DataFrame({'a':[1.,np.nan,5.,np.nan],'b':[np.nan,2.,np.nan,6.],'c':range(2,18,4)})

In [109]:
df2=pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],'b':[np.nan,3.,4.,6.,8.]})

In [110]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [111]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [112]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [113]:
#8.3重塑和轴向旋转
#重塑层次化索引
#层次化索引为DataFrame数据的重排任务
data=pd.DataFrame(np.arange(6).reshape((2,3)),index=pd.Index(['Ohio','Colorado'],name='state'),columns=pd.Index(['one','two','three'],name='number'))

In [114]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [115]:
result=data.stack()

In [116]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [117]:
#对一个层次化索引的Series，可以用unstack将其重排为一个DataFrame
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [118]:
#默认情况下，unstack操作的是最内层。传入分层级别的编号或名称即可对其他级别进行unstack操作
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [119]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [120]:
#如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能回引入缺失数据
s1=pd.Series([0,1,2,3],index=['a','b','c','d'])

In [121]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [122]:
s2=pd.Series([4,5,6],index=['c','d','e'])

In [123]:
data2=pd.concat([s1,s2],keys=['one','two'])

In [124]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [125]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [126]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [127]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [128]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [130]:
#在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别
df=pd.DataFrame({'left':result,'right':result+5},columns=pd.Index(['left','right'],
                                                                  name='side'))
df#324

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [131]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [132]:
#当调用stack，我们可以指明轴的名字：
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [133]:
#将‘长格式’旋转为‘宽格式’
#多个时间序列数据通常是以所谓的‘长格式’（long）或‘堆叠格式’存储在数据库和csv中
#将‘宽格式’旋转为‘长格式’


In [134]:
#旋转DataFrame的逆运算是pandas.melt.它不是将一列转换到多个新的DataFrame，
#而是合并多个列称为一个，产生一个比输入长的DataFrame
df=pd.DataFrame({'key':['foo','bar','baz'],'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})

In [135]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [136]:
#key列可能是分组指标，其他列是数据值。当使用pandas.melt,我们必须指明那些列是分组指标。
melted=pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [137]:
#使用pivot，可以重塑回原来的样子
reshaped=melted.pivot('key','variable','value')

In [138]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [139]:
#pivot的结果从列创建了一个索引，用作行标签，我们可以使用reset_idnex将数据移回列
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [140]:
#还可以指定列的子集，作为值得列：
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [141]:
#pandas.melt也可以不用分组指标：
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [142]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [ ]:
#8.4总结，下一章学习matplotlib数据可视化